# Fake News Classification with the help of Natural Language Processing Technique.

Fake news detection is a controversial topic in the field of natural language processing. People consume news through several mediums throughout the day in their daily routine, but sometimes it becomes difficult to decide which one is fake and which one is authentic. The aim of this project is to create a model which predicts whether a given news is real or fake.

# Required Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1.Data Gathering

In [2]:
news_df = pd.read_csv('/content/News_dataset.csv')
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# 2. Data Analysis

In [3]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [4]:
news_df['label'].value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

In [5]:
news_df.shape

(20800, 5)

In [6]:
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
news_df = news_df.dropna() # dropping rows which have missing values

In [8]:
news_df.shape

(18285, 5)

In [9]:
news_df.reset_index(inplace=True)

In [10]:
news_df.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
news_df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [12]:
news_df = news_df.drop(['id', 'text', 'author'], axis=1)
news_df.head()

,index,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,2,Why the Truth Might Get You Fired,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,1
4,4,Iranian woman jailed for fictional unpublished...,1


# 3. Data Preprocessing

In [13]:
sample_text = 'The black car should be very same to the white car '
sample_text = sample_text.split()

In [14]:
sample_text = [text.lower() for text in sample_text]
sample_text

['the',
 'black',
 'car',
 'should',
 'be',
 'very',
 'same',
 'to',
 'the',
 'white',
 'car']

In [15]:
stop_words = stopwords.words('english')
print(stop_words[0:10])
print(len(stop_words))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
179


In [16]:
sample_text = [text for text in sample_text if text not in stop_words]
print(sample_text)
len(sample_text)

['black', 'car', 'white', 'car']


4

In [17]:
ps = PorterStemmer()
sample_text_stemming = [ps.stem(text) for text in sample_text]
print(sample_text_stemming)

['black', 'car', 'white', 'car']


In [18]:
lm = WordNetLemmatizer()
sample_text_lemma = [lm.lemmatize(text) for text in sample_text]
print(sample_text_lemma)

['black', 'car', 'white', 'car']


In [ ]:
lm = WordNetLemmatizer()
corpus = []
for i in range(len(news_df)):
    review = re.sub('^a-zA-Z0-9', '', news_df['title'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(x) for x in review if x not in stop_words]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
len(corpus)

18285

In [21]:
corpus[-1]

'keep f-35 alive'

# 4. Vectorization (Convert Text data into the Vector)

In [22]:
tf = TfidfVectorizer()
X = tf.fit_transform(corpus).toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
y = news_df['label']
y.tail()

18280    0
18281    0
18282    0
18283    1
18284    1
Name: label, dtype: int64

## Data splitting into the train and test

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=22)

In [25]:
print(len(X_train), len(X_test))

12250 6035


In [26]:
print(len(y_train), len(y_test))

12250 6035


# 5. Building Model

In [27]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

# 6. Model Evaluation

In [28]:
y_prediction = rf.predict(X_test)
acc_score = accuracy_score(y_test, y_prediction)
acc_score

0.9338856669428335

In [29]:
class Evaluation:
    def __init__(self, model, X_train, X_test, y_train, y_test):
        self.model = model
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

    def train_evaluation(self):
        y_pred_train = self.model.predict(self.X_train)
        acc_score_train = accuracy_score(self.y_train, y_pred_train)
        print('Accuracy Score on training Dataset:', acc_score_train)
        print()

        con_mat_train = confusion_matrix(self.y_train, y_pred_train)
        print("Confusion Matrix On training dataset :\n",con_mat_train)
        print()

        class_rep_train = classification_report(self.y_train, y_pred_train)
        print('Classification Report on trainining dataset:\n', class_rep_train)

    def test_evaluation(self):
        y_pred_test = self.model.predict(self.X_test)
        acc_score_test = accuracy_score(self.y_test, y_pred_test)
        print('Accuracy Score on testing dataset:', acc_score_test)
        print()

        con_mat_test = confusion_matrix(self.y_test, y_pred_test)
        print("Confusion Matrix On testing dataset :\n",con_mat_test)
        print()

        class_rep_test = classification_report(self.y_test, y_pred_test)
        print('Classification Report on testing dataset:\n', class_rep_test)

In [30]:
Evaluation(rf, X_train, X_test, y_train, y_test).train_evaluation()

Accuracy Score on training Dataset: 1.0

Confusion Matrix On training dataset :
 [[6941    0]
 [   0 5309]]

Classification Report on trainining dataset:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6941
           1       1.00      1.00      1.00      5309

    accuracy                           1.00     12250
   macro avg       1.00      1.00      1.00     12250
weighted avg       1.00      1.00      1.00     12250



In [31]:
Evaluation(rf,X_train, X_test, y_train, y_test).test_evaluation()

Accuracy Score on testing dataset: 0.9338856669428335

Confusion Matrix On testing dataset :
 [[3076  344]
 [  55 2560]]

Classification Report on testing dataset:
               precision    recall  f1-score   support

           0       0.98      0.90      0.94      3420
           1       0.88      0.98      0.93      2615

    accuracy                           0.93      6035
   macro avg       0.93      0.94      0.93      6035
weighted avg       0.94      0.93      0.93      6035



# Prediction Pipeline

In [32]:
news_df['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [33]:
class Preprocessing:

    def __init__(self, data):
        self.data = data
        # Fetch the stopwords list and convert it to a set for faster lookups
        self.stopwords = set(stopwords.words('english'))
        self.lm = WordNetLemmatizer()

    def text_preprocessing_user(self):
        # Remove non-alphanumeric characters (keeping spaces)
        review = re.sub(r'[^a-zA-Z0-9\s]', '', self.data)
        review = review.lower()
        review = review.split()
        # Lemmatize and filter out stopwords
        review = [self.lm.lemmatize(x) for x in review if x not in self.stopwords]
        review = " ".join(review)
        return review

In [34]:
class Prediction:

    def __init__(self, pred_data, model, tfidf_transformer):
        self.pred_data = pred_data
        self.model = model
        self.tfidf_transformer = tfidf_transformer

    def prediction_model(self):
        preprocess_data = Preprocessing(self.pred_data).text_preprocessing_user()
        data = self.tfidf_transformer.transform([preprocess_data])
        prediction = self.model.predict(data)

        if prediction[0] == 0:
            return "The News Is Fake"
        else:
            return "The News Is Real"


In [35]:
news = 'SETI officially announce Extraterrestrial signals'
Prediction(news,rf, tf).prediction_model()

'The News Is Real'